In [30]:
# api 불러오기
import os
from dotenv import load_dotenv
load_dotenv()

# 환경 변수 불러오기
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
LANGCHAIN_TRACING_V2 = os.getenv("LANGCHAIN_TRACING_V2")
LANGCHAIN_ENDPOINT = os.getenv("LANGCHAIN_ENDPOINT")
LANGCHAIN_PROJECT = os.getenv("LANGCHAIN_PROJECT")

In [31]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

llm = ChatOpenAI(temperature=0, model_name="gpt-4o")

In [32]:
email_conversation = """
From: John (John@bikecorporation.me)
To: Kim (Kim@teddyinternational.me)
Subject: “ZENESIS” bike distribution cooperation and meeting schedule proposal
Dear Mr. Kim,

I am John, Senior Executive Director at Bike Corporation. I recently learned about your new bicycle model, "ZENESIS," through your press release. Bike Corporation is a company that leads innovation and quality in the field of bicycle manufacturing and distribution, with long-time experience and expertise in this field.

We would like to request a detailed brochure for the ZENESIS model. In particular, we need information on technical specifications, battery performance, and design aspects. This information will help us further refine our proposed distribution strategy and marketing plan.

Additionally, to discuss the possibilities for collaboration in more detail, I propose a meeting next Tuesday, January 15th, at 10:00 AM. Would it be possible to meet at your office to have this discussion?

Thank you.

Best regards,
John
Senior Executive Director
Bike Corporation
"""

# oupput parser 사용하지 않는경우

In [33]:
from itertools import chain
from langchain_core.prompts import PromptTemplate
from langchain_core.messages import AIMessageChunk
from langchain_core.output_parsers import StrOutputParser
from itertools import chain
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    "다음의 이메일 내용중 중요한 내용을 추출해 주세요.\n\n{email_conversation}"
)

llm = ChatOpenAI(temperature=0, model_name="gpt-4o")

chain = prompt | llm

answer = chain.stream({"email_conversation": email_conversation})

In [34]:
def stream_response(response, return_output=False):
    """
    Streams the response from the AI model, processing and printing each chunk.

    This function iterates over each item in the 'response' iterable. If an item is an instance of AIMessageChunk, it extracts and prints the content.
    If the item is a string, it prints the string directly.
    Optionally, the function can return the concatenated string of all response chunks.

    Args:
    - response (iterable): An iterable of response chunks, which can be AIMessageChunk objects or strings.
    - return_output (bool, optional): If True, the function returns the concatenated response string. The default is False.

    Returns:
    - str: If `return_output` is True, the concatenated response string. Otherwise, nothing is returned.
    """
    answer = ""
    for token in response:
        if isinstance(token, AIMessageChunk):
            answer += token.content
            print(token.content, end="", flush=True)
        elif isinstance(token, str):
            answer += token
            print(token, end="", flush=True)
    if return_output:
        return answer
    
output = stream_response(answer, return_output=True)

Here are the important points from the email:

1. John, Senior Executive Director at Bike Corporation, is interested in the "ZENESIS" bicycle model from Teddy International.
2. He requests a detailed brochure for the ZENESIS model, focusing on technical specifications, battery performance, and design aspects.
3. John proposes a meeting to discuss potential collaboration on Tuesday, January 15th, at 10:00 AM, at Kim's office.

In [35]:
!pip install langchain_core

In [36]:
from itertools import chain
from langchain_core.prompts import PromptTemplate
from langchain_core.messages import AIMessage
from langchain_core.output_parsers import StrOutputParser

prompt = PromptTemplate.from_template(
    """Please extract the important parts of the following email.
    {email_conversation}
"""
)

llm = ChatOpenAI(temperature=0, model_name="gpt-4o")
chain = prompt | llm
answer = chain.stream({"email_conversation":email_conversation})
output = stream_response(answer, return_output=True)

- **Sender:** John (Senior Executive Director at Bike Corporation)
- **Recipient:** Kim (Teddy International)
- **Subject:** "ZENESIS" bike distribution cooperation and meeting schedule proposal
- **Key Points:**
  - John learned about the "ZENESIS" bicycle model through a press release.
  - Bike Corporation is interested in a detailed brochure for the ZENESIS model, focusing on technical specifications, battery performance, and design aspects.
  - John proposes a meeting to discuss collaboration possibilities in more detail.
  - Suggested meeting date and time: Next Tuesday, January 15th, at 10:00 AM, at Kim's office.

In [37]:
class EmailSummary(BaseModel):
    person: str = Field(description="메일을 보낸 사람")
    email: str = Field(description="메일을 보낸 사람의 이메일 주소")
    subject: str = Field(description="메일 제목")
    summary: str = Field(description="메일 본문을 요약한 텍스트")
    Is_Spam: str = Field(description="스팸 메일 여부")
    date: str = Field(description="메일 본문에 언급된 미팅 날짜와 시간")

In [38]:
# PydanticOutputParser 생성
parser = PydanticOutputParser(pydantic_object=EmailSummary)

In [39]:
prompt = PromptTemplate.from_template(
    """
You are a helpful assistant. Please answer the following questions in KOREAN.

QUESTION:
{question}

EMAIL CONVERSATION:
{email_conversation}

FORMAT:
{format}
"""
)

In [40]:
# format 에 PydanticOutputParser의 부분 포맷팅(partial) 추가
prompt = prompt.partial(format=parser.get_format_instructions())


In [41]:
prompt

PromptTemplate(input_variables=['email_conversation', 'question'], input_types={}, partial_variables={'format': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"person": {"description": "메일을 보낸 사람", "title": "Person", "type": "string"}, "email": {"description": "메일을 보낸 사람의 이메일 주소", "title": "Email", "type": "string"}, "subject": {"description": "메일 제목", "title": "Subject", "type": "string"}, "summary": {"description": "메일 본문을 요약한 텍스트", "title": "Summary", "type": "string"}, "date": {"description": "메일 본문에 언급된 미팅 날짜와 시간", "title": "Date", "type": "string"}}, "required

In [42]:
# chain 을 생성합니다.
chain = prompt | llm

In [43]:
# chain 을 실행하고 결과를 출력합니다.
response = chain.stream(
    {
        "email_conversation": email_conversation,
        "question": "이메일 내용중 주요 내용을 추출해.",
    }
)

In [44]:
# 결과는 JSON 형태로 출력됩니다.
output = stream_response(response, return_output=True)

```json
{
    "person": "John",
    "email": "John@bikecorporation.me",
    "subject": "“ZENESIS” bike distribution cooperation and meeting schedule proposal",
    "summary": "John은 Bike Corporation의 고위 임원으로, Kim에게 ZENESIS 자전거 모델의 상세 브로셔를 요청하고, 협력 가능성을 논의하기 위해 1월 15일 화요일 오전 10시에 Kim의 사무실에서 회의를 제안합니다.",
    "date": "January 15th, 10:00 AM"
}
```

In [45]:
# chain 을 실행하고 결과를 출력합니다.
response = chain.invoke(
    {
        "email_conversation": email_conversation,
        "question": "이메일 내용중 주요 내용을 추출해 주세요.",
    }
)

# 결과는 EmailSummary 객체 형태로 출력됩니다.
print(response)

content='```json\n{\n    "person": "John",\n    "email": "John@bikecorporation.me",\n    "subject": "“ZENESIS” bike distribution cooperation and meeting schedule proposal",\n    "summary": "John은 Bike Corporation의 고위 임원으로, Kim에게 ZENESIS 자전거 모델의 상세 브로셔를 요청하며, 기술 사양, 배터리 성능, 디자인 측면에 대한 정보를 필요로 한다고 언급했다. 또한, 협력 가능성을 논의하기 위해 1월 15일 화요일 오전 10시에 Kim의 사무실에서 회의를 제안했다.",\n    "date": "January 15th, 10:00 AM"\n}\n```' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 151, 'prompt_tokens': 530, 'total_tokens': 681, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_4691090a87', 'finish_reason': 'stop', 'logprobs': None} id='run-5e0639fb-a47a-48ee-bf9f-a37d5dfbd431-0' usage_metadata={'input_tokens': 530, 'output_tokens': 151, 'total_tokens': 68

In [49]:
response.person

AttributeError: 'AIMessage' object has no attribute 'person'